In [2]:
from pybrain.tools.shortcuts import buildNetwork
from pybrain.datasets import SupervisedDataSet
from pybrain.supervised.trainers import BackpropTrainer
import pandas as pd

numIn = 30 #how many time steps are we looking at
numHidden = 3 #number of hidden layers
numOut = 1 # len(apps) * 2 (on/off?)

In [3]:
net = buildNetwork(numIn, numHidden, numOut)
ds = SupervisedDataSet(numIn,numOut)
trainer = BackpropTrainer(net, ds)

def train():
    global net, ds, trainer
    print trainer.train()
    
def toWindow(data):
    for r in range(0, len(laptop)):
        yield laptop['current'][r:r+numIn]

In [4]:
laptop = pd.read_csv('laptop.csv')
laptop.head()
laptop['time'] = laptop['time'].map( lambda t: float(t - laptop['time'][0]) * 0.001) #convert to milliseconds starting at 0
laptop_inputs = pd.Series([li for li in toWindow(laptop) if len(li) == numIn])
laptop_outputs = [0] * len(laptop_inputs)
laptop_outputs[584:1138] = [1] * len(laptop_outputs[584:1138])
laptop_outputs[4150:4242] = [-1] * len(laptop_outputs[4150:4242])
laptop_inputs.head()

0    0     0
1     2
2     2
3     3
4     0
5     ...
1    1     2
2     2
3     3
4     0
5     2
6     ...
2    2     2
3     3
4     0
5     2
6     0
7     ...
3    3     3
4     0
5     2
6     0
7     1
8     ...
4    4     0
5     2
6     0
7     1
8     0
9     ...
dtype: object

In [5]:
for i in range(len(laptop_outputs)):
    ds.addSample(laptop_inputs[i], laptop_outputs[i])

In [6]:
#epoch_number = 10
#for i in range(epoch_number):
#    train()
trainer.trainUntilConvergence()

KeyboardInterrupt: 

In [10]:
print len(laptop_inputs[0])
training_ds = laptop_inputs.tolist()
def test(training_ds):
    global net
    lst = []
    for training in training_ds:
        lst.append(net.activate(training))
    return lst
test_ans = pd.Series(test(training_ds))

30


In [ ]:
norm_ans = []
for ans in test_ans:
    if ans > 0.2:
        norm_ans.append(1)
    elif ans > 0:
        norm_ans.append(0)
    else:
        norm_ans.append(-1)
norm_ans = pd.Series(norm_ans)
print float(sum([1 if norm_ans[i] == laptop_outputs[i] else 0 for i,_ in enumerate(norm_ans)])) / float(len(norm_ans))
print float(sum([1 if 0 == laptop_outputs[i] else 0 for i,_ in enumerate(norm_ans)])) / float(len(norm_ans))

In [ ]:
for x in zip(norm_ans.tolist(), laptop_outputs):
    print x